In [ ]:
################################################################################################################

# EXTERNAL MODULES TO BE USED

################################################################################################################

from flask import Flask, render_template, redirect, url_for, request   
from flask_wtf import FlaskForm
from wtforms import StringField, PasswordField, SubmitField
from wtforms.validators import InputRequired, Email, Length, ValidationError
from flask_sqlalchemy import SQLAlchemy 
from flask_login import LoginManager, UserMixin, login_user, login_required, logout_user, current_user
from flask_bcrypt import Bcrypt
import pandas as pd
from datetime import datetime
app = Flask(__name__)

################################################################################################################

# WEB ROUTES FOR CONTROLLING ACCESS TO TEMPLATE VIEWS

################################################################################################################

app = Flask(__name__)
app.config['SECRET_KEY'] = 'thisisasecretkey'
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///database.db'
db = SQLAlchemy(app)
bcrypt = Bcrypt(app)

login_manager = LoginManager()
login_manager.init_app(app)
login_manager.login_message = "You need to login again."
login_manager.refresh_view = 'login'
login_manager.login_view = 'login'
   

class Users(db.Model, UserMixin):
    id = db.Column(db.Integer, primary_key=True)
    username = db.Column(db.String(15), unique=True)
    email = db.Column(db.String(50), unique=True)
    password = db.Column(db.String(80))
    
    def __repr__(self):
        return '<User %r>' %self.id

db.create_all()

    
@login_manager.user_loader
def load_user(user_id):
    return Users.query.get(int(user_id))

class RegisterForm(FlaskForm):
    email = StringField("Email", validators=[InputRequired(), Email(message="Invalid Email"), Length(max=50)], render_kw={"placeholder": "example@gmail.com"})
    username = StringField("Username", validators=[InputRequired(), Length(min=4, max=15)], render_kw={"placeholder": "Username"})
    password = PasswordField("Password", validators=[InputRequired(), Length(min=4, max=15)], render_kw={"placeholder": "********"})
    submit = SubmitField("Sign Up")

    def validate_username(self, username):
        existing_user_username = Users.query.filter_by(username=username.data).first()
        if existing_user_username:
            raise ValidationError("That username already exists. Please choose a different one.")

    def validate_email(self, email):
        existing_user_email = Users.query.filter_by(email=email.data).first()
        if existing_user_email:
            raise ValidationError("That email address belongs to different user. Please choose a different one.")

class LoginForm(FlaskForm):
    username = StringField('Username', validators=[InputRequired()])
    password = PasswordField('Password', validators=[InputRequired()])
    submit = SubmitField('Login')

@app.route("/")
def index():
    return render_template('index.html')

@app.route("/about")
def about():
    return render_template('about.html')


@app.route("/post")
def post():
    return render_template('post.html')


@app.route("/contact")
def contact():
    return render_template('contact.html')

    

@app.route('/register', methods=['GET','POST'])
def register():
    form = RegisterForm()
    if form.validate_on_submit():
        hashed_password = bcrypt.generate_password_hash(form.password.data)
        new_user = Users(username=form.username.data, email=form.email.data, password=hashed_password)
        db.session.add(new_user)
        db.session.commit()
        return redirect(url_for('login'))
    return render_template('register.html', title='Register', form=form)

@app.route('/login', methods=['GET','POST'])
def login():
    form = LoginForm()
    if form.validate_on_submit():
        user = Users.query.filter_by(username=form.username.data).first()
        if user:
            if bcrypt.check_password_hash(user.password, form.password.data):
                login_user(user)
                return redirect(url_for("index"))
    return render_template('flask_login.html', title="Login", form=form)

@app.route("/logout")
@login_required
def logout():
    print('logout user.')
    logout_user()
    return redirect(url_for('index')) 


@app.route("/store")
def store():
    return render_template('store.html')


################################################################################################################

# ERROR HANDLERS

################################################################################################################

@app.errorhandler(404)
def page_not_found(e):
    return render_template('404.html'), 404

################################################################################################################

# APPLICATION TEST RUN AT PORT 9002

################################################################################################################

if __name__ == '__main__':
    app.run('localhost', 9002)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


/opt/anaconda3/lib/python3.8/site-packages/flask_sqlalchemy/__init__.py:872: FSADeprecationWarning: SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and will be disabled by default in the future.  Set it to True or False to suppress this warning.
  warnings.warn(FSADeprecationWarning(
 * Running on http://localhost:9002/ (Press CTRL+C to quit)
127.0.0.1 - - [15/Nov/2021 13:58:11] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [15/Nov/2021 13:58:15] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [15/Nov/2021 13:58:17] "GET /post HTTP/1.1" 200 -
127.0.0.1 - - [15/Nov/2021 13:58:17] "GET /static/assets/img/blog-lp.jpg HTTP/1.1" 304 -
127.0.0.1 - - [15/Nov/2021 13:58:22] "GET /about HTTP/1.1" 200 -
127.0.0.1 - - [15/Nov/2021 13:58:22] "GET /static/assets/img/about-lp.jpg HTTP/1.1" 304 -
127.0.0.1 - - [15/Nov/2021 13:58:24] "GET /contact HTTP/1.1" 200 -
127.0.0.1 - - [15/Nov/2021 13:58:25] "GET /static/assets/mail/contact_me.js HTTP/1.1" 304 -
127.0.0.1 - - [15/Nov/2021 13:58:25] "GET /static/assets/ma

logout user.


127.0.0.1 - - [15/Nov/2021 14:00:31] "GET /login HTTP/1.1" 200 -
127.0.0.1 - - [15/Nov/2021 14:00:33] "GET /contact HTTP/1.1" 200 -
127.0.0.1 - - [15/Nov/2021 14:00:33] "GET /about HTTP/1.1" 200 -
127.0.0.1 - - [15/Nov/2021 14:00:35] "GET /post HTTP/1.1" 200 -
127.0.0.1 - - [15/Nov/2021 14:00:38] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [15/Nov/2021 14:00:41] "GET /index.html HTTP/1.1" 404 -
127.0.0.1 - - [15/Nov/2021 14:00:41] "GET /static/assets/img/backup-lp.jpg HTTP/1.1" 304 -
127.0.0.1 - - [15/Nov/2021 14:00:42] "GET / HTTP/1.1" 200 -
